<a href="https://colab.research.google.com/github/karina-ferreira/pensamento_computacional_JDAD/blob/main/projeto_final_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- *Passo 1:* O robô vai fazer uma solicitação para a API da Wikipédia, usando como parâmetro a página de revisões de um verbete que eu indicar.

- *Passo 2:* Do dicionário que ele trouxer, preciso separar os seguintes dados: data da revisão, usuário que editou, se aumentou ou diminuiu o tamanho do arquivo (em caracteres). Transformar data em datetime, conferir se o tamanho é string ou número. Montar um dataframe. 

- *Passo 3:* O programa também vai mostrar quantas edições já foram feitas no verbete e por quantos usuários. No caso da mesma solicitação ter vários verbetes de uma vez, vou classificá-las em ordem crescente de quem mais teve edições num período que eu definir. 

- *Passo 4:* Salvar em um excel. 

- *Extra:* Notificar quando algum verbete que eu estiver monitorando (rodando o código 1x por semana) for editado x% mais vezes ou tiver x% de seu tamanho alterado do que na semana anterior. Tuitando?  

In [99]:
import requests
import pandas as pd

In [100]:
BASE_URL = 'https://pt.wikipedia.org/w/api.php' # para outros idiomas, é preciso mudar a abreviatura no link. Estou usando 'pt'

parameters = {
          # 'pageids': 315393, # está nos três risquinhos, "informações da página"
          'titles': 'Guilherme_Peirão_Leal', # retirado do link do verbete
          'action': 'query', 
          'prop': 'revisions',
          'rvprop': 'content|size|ids|timestamp|sha1|comment|flags|user|userid',
          'rvlimit': 'max', 
          'format': 'json', 
          'rvdir': 'older'}

result = requests.get(url=BASE_URL, params=parameters).json()

O programa mostra apenas as últimas 50 revisões por verbete. Para contarmos todas, é preciso pegar o valor contido em `rvcontinue`, que é uma espécie de chave para continuar.

In [101]:
import requests

BASE_URL = "http://pt.wikipedia.org/w/api.php"


parameters = { 'action': 'query',
           'format': 'json',
           'continue': '',
           'titles': 'Guilherme_Peirão_Leal',
          #  'pageids': 315393,
           'prop': 'revisions',
           'rvprop': 'ids|userid',
           'rvlimit': 'max'}

wp_call = requests.get(BASE_URL, params=parameters)
response = wp_call.json()

total_revisions = 0

while True:
  wp_call = requests.get(BASE_URL, params=parameters)
  response = wp_call.json()

  for page_id in response['query']['pages']:
    total_revisions += len(response['query']['pages'][page_id]['revisions'])

  if 'continue' in response:
    parameters['continue'] = response['continue']['continue']
    parameters['rvcontinue'] = response['continue']['rvcontinue']

  else:
    break

print (total_revisions)

93


Agora preciso transformar a lista de dicionários que ele devolve (onde cada dicionário é uma revisão) em um dataframe. Os dicionários têm as mesmas chaves, logo, posso tratá-los com JSON?

In [102]:
revisoes = result['query']['pages']['315393']['revisions']

In [103]:
revisoes

[{'revid': 64753229,
  'parentid': 62524821,
  'user': 'Karina dos Santos Ferreira',
  'userid': 2836954,
  'timestamp': '2022-11-16T17:05:06Z',
  'size': 14755,
  'sha1': '29ae40705ba4a9ee94c0d051217fe091c5660e87',
  'contentformat': 'text/x-wiki',
  'contentmodel': 'wikitext',
  'comment': 'Dados desatualizados como porcentagem de ações na Natura; organizações da sociedade civil criadas por ele; a definição como "político" tendo em vista que atuou há 10 anos e apenas por um período de 2 anos.',
  '*': '{{Info/Político\n|nome                  = Guilherme Leal\n|nome_alt              = \n|imagem                = \n|imagem_legenda        = \n|data_nascimento       = {{dni|lang=br|22|2|1950}}\n|local_nascimento      = [[Santos]] ([[São Paulo (estado)|SP]])\n|data_morte            = \n|local_morte           = \n|nacionalidade         = {{BRAn|o}}\n|nome_mãe              = \n|nome_pai              = \n|casamento_progenit    = \n|tipo-conjuge          = Esposa\n|conjuge               = \n|r

In [104]:
df = pd.read_json(revisoes) # ele diz que não consegue guardar um JSON em uma variável, somente se fosse um arquivo JSON em uma variável.

ValueError: ignored

In [106]:
import json

In [107]:
rev_json = json.dumps(revisoes) # forma de transformar a variável em um >>objeto<< JSON

In [108]:
from io import StringIO # dava um erro, que a internet disse para solucionar com esse método

In [109]:
df_revisoes = pd.read_json(StringIO(rev_json))

In [110]:
df_revisoes.head()

,revid,parentid,user,userid,timestamp,size,sha1,contentformat,contentmodel,comment,*,minor,anon
0,64753229,62524821,Karina dos Santos Ferreira,2836954,2022-11-16 17:05:06+00:00,14755,29ae40705ba4a9ee94c0d051217fe091c5660e87,text/x-wiki,wikitext,Dados desatualizados como porcentagem de ações...,{{Info/Político\n|nome = Guil...,NaN,NaN
1,62524821,59508398,FelipeKrol,2312030,2021-12-01 22:53:52+00:00,11758,273d68c988029462070802e5dd306b4f6e77f32c,text/x-wiki,wikitext,,{{Info/Político\n|nome =Guilherme...,,NaN
2,59508398,58000364,OnlyJonny,588631,2020-10-03 15:56:19+00:00,11649,8ff7e00f480e65dfc83d47835e3df70e0b29c6f5,text/x-wiki,wikitext,"ajustes gerais nas citações, outros ajustes us...",{{Info/Político\n|nome =Guilherme...,,NaN
3,58000364,57753780,Aleth Bot,862253,2020-04-10 07:01:30+00:00,11657,acbf3f5c8702cefd046db24d95ed7ad0039a4880,text/x-wiki,wikitext,[[Wikipédia:Bot|Bot]]: Substituição automática...,{{Info/Político\n|nome =Guilherme...,,NaN
4,57753780,57753597,Douglasboavista,1412904,2020-03-09 18:58:41+00:00,11648,3c3adefaed76533db058952f2b485d9f60d6686a,text/x-wiki,wikitext,Foram [[WP:REV|revertidas]] as edições de [[Sp...,{{Info/Político\n|nome =Guilherme...,,NaN


In [111]:
df_revisoes.columns

Index(['revid', 'parentid', 'user', 'userid', 'timestamp', 'size', 'sha1',
       'contentformat', 'contentmodel', 'comment', '*', 'minor', 'anon'],
      dtype='object')

In [112]:
df_revisoes.drop(columns=['revid', 'parentid', 'userid', 'sha1', 'contentformat', 'contentmodel', 'minor', 'anon'], inplace=True)

In [113]:
df_revisoes.head()

,user,timestamp,size,comment,*
0,Karina dos Santos Ferreira,2022-11-16 17:05:06+00:00,14755,Dados desatualizados como porcentagem de ações...,{{Info/Político\n|nome = Guil...
1,FelipeKrol,2021-12-01 22:53:52+00:00,11758,,{{Info/Político\n|nome =Guilherme...
2,OnlyJonny,2020-10-03 15:56:19+00:00,11649,"ajustes gerais nas citações, outros ajustes us...",{{Info/Político\n|nome =Guilherme...
3,Aleth Bot,2020-04-10 07:01:30+00:00,11657,[[Wikipédia:Bot|Bot]]: Substituição automática...,{{Info/Político\n|nome =Guilherme...
4,Douglasboavista,2020-03-09 18:58:41+00:00,11648,Foram [[WP:REV|revertidas]] as edições de [[Sp...,{{Info/Político\n|nome =Guilherme...


In [114]:
df_revisoes['timestamp'] = pd.to_datetime(df_revisoes['timestamp']) # transformando a coluna timestamp em objeto datetime

In [115]:
df_revisoes = df_revisoes.sort_values("timestamp", ascending=False) # ordenando da mais recente para a mais antiga

In [116]:
df_revisoes.head()

,user,timestamp,size,comment,*
0,Karina dos Santos Ferreira,2022-11-16 17:05:06+00:00,14755,Dados desatualizados como porcentagem de ações...,{{Info/Político\n|nome = Guil...
1,FelipeKrol,2021-12-01 22:53:52+00:00,11758,,{{Info/Político\n|nome =Guilherme...
2,OnlyJonny,2020-10-03 15:56:19+00:00,11649,"ajustes gerais nas citações, outros ajustes us...",{{Info/Político\n|nome =Guilherme...
3,Aleth Bot,2020-04-10 07:01:30+00:00,11657,[[Wikipédia:Bot|Bot]]: Substituição automática...,{{Info/Político\n|nome =Guilherme...
4,Douglasboavista,2020-03-09 18:58:41+00:00,11648,Foram [[WP:REV|revertidas]] as edições de [[Sp...,{{Info/Político\n|nome =Guilherme...


- Fazer o `len`da revisão em si, já que a API não me trouxe esse dado 😠;
- Escrever o código para o que fazer quando entrar mais de um verbete na mesma consulta;
- Automatizar as análises:
     * ***Nos últimos 7 dias, o verbete:***
    - foi editado `x` vezes. Isso é `x%` a mais que na semana anterior, e `x%` a mais que no período monitorado (`entre xx/xx/xxxx e hoje`);
    - aumentou/diminuiu de tamanho em `x%`;
    - foi editado por `nº` usuários.
- Salvar em CSV;
- Acrescentar os novos dados (toda vez que ele rodar) no mesmo CSV.

In [117]:
df_revisoes['tamanho'] = df_revisoes['*'].apply(lambda x: len(x))

In [118]:
df_revisoes.head()

,user,timestamp,size,comment,*,tamanho
0,Karina dos Santos Ferreira,2022-11-16 17:05:06+00:00,14755,Dados desatualizados como porcentagem de ações...,{{Info/Político\n|nome = Guil...,14469
1,FelipeKrol,2021-12-01 22:53:52+00:00,11758,,{{Info/Político\n|nome =Guilherme...,11513
2,OnlyJonny,2020-10-03 15:56:19+00:00,11649,"ajustes gerais nas citações, outros ajustes us...",{{Info/Político\n|nome =Guilherme...,11407
3,Aleth Bot,2020-04-10 07:01:30+00:00,11657,[[Wikipédia:Bot|Bot]]: Substituição automática...,{{Info/Político\n|nome =Guilherme...,11415
4,Douglasboavista,2020-03-09 18:58:41+00:00,11648,Foram [[WP:REV|revertidas]] as edições de [[Sp...,{{Info/Político\n|nome =Guilherme...,11406


Fazendo a nova coluna que mostra a diferença de caracteres entre essa e a última edição:

In [119]:
df_revisoes['tamanho_anterior'] = df_revisoes['tamanho'].shift(-1)

In [120]:
df_revisoes['tamanho_edicao'] = df_revisoes['tamanho'] - df_revisoes['tamanho_anterior']

In [121]:
df_revisoes.head()

,user,timestamp,size,comment,*,tamanho,tamanho_anterior,tamanho_edicao
0,Karina dos Santos Ferreira,2022-11-16 17:05:06+00:00,14755,Dados desatualizados como porcentagem de ações...,{{Info/Político\n|nome = Guil...,14469,11513.0,2956.0
1,FelipeKrol,2021-12-01 22:53:52+00:00,11758,,{{Info/Político\n|nome =Guilherme...,11513,11407.0,106.0
2,OnlyJonny,2020-10-03 15:56:19+00:00,11649,"ajustes gerais nas citações, outros ajustes us...",{{Info/Político\n|nome =Guilherme...,11407,11415.0,-8.0
3,Aleth Bot,2020-04-10 07:01:30+00:00,11657,[[Wikipédia:Bot|Bot]]: Substituição automática...,{{Info/Político\n|nome =Guilherme...,11415,11406.0,9.0
4,Douglasboavista,2020-03-09 18:58:41+00:00,11648,Foram [[WP:REV|revertidas]] as edições de [[Sp...,{{Info/Político\n|nome =Guilherme...,11406,11509.0,-103.0


Preciso adicionar os dados que eu obter cada vez que rodo o código à mesma tabela que vou criar para aquele verbete/consulta. Ex a seguir:

In [122]:
df_revisoes.to_csv('revisoes.csv') # salvar o dataframe em csv

In [123]:
df_revisoes.to_csv('revisoes.csv', sep=';', mode = 'a', header = False) # salvar novas linhas sem cabeçalho